In [1]:
import torch
import numpy as np
from utils.generate_csbm import *
from utils.train import *
from utils.utils import *
from models.models import *
from models.setup import *
import networkx as nx
import pickle

In [2]:
def exp_csbm(ith, method, folder_path, csbm, n_train, n_val, 
             num_communication=100, batch_size=1,
             learning_rate=0.01, I=10, gradient=True, noise=False,
             Print=False, print_time=1,
             resume=False,
             bias=False,
             save=True):
    

    torch.manual_seed(0)
    init_mlp = MLP(csbm.Xs[0].shape[1], 64, 2, bias)
        
    if (gradient == False):
        grad = "no_grad/"
    elif (gradient == True and noise==False):
        grad = "biased_grad/"
    elif (gradient == True and noise==True):
        grad = "noisy_grad/"
      
    A_tilde = calculate_Atilde(csbm.A, K=10, alpha=0.9)
    server = set_up_GC(csbm.Xs, csbm.ys, init_mlp, A_tilde, n_train=n_train, n_val=n_val)
   
        
    tl, ta, vl, va = train_GC(server, num_communication, batch_size, learning_rate, I,
                              gradient, noise, 
                              Print, print_time)
    if save:    
        np.save(folder_path + method + grad + "I" + str(I) + "/tl_" + str(ith), tl)
        np.save(folder_path + method + grad + "I" + str(I) + "/ta_" + str(ith), ta)
        np.save(folder_path + method + grad + "I" + str(I) + "/vl_" + str(ith), vl)
        np.save(folder_path + method + grad + "I" + str(I) + "/va_" + str(ith), va)

        PATH = folder_path + method + grad + "I" + str(I) + "/model_" + str(ith)
        torch.save({
                'best_model_state_dict': server.best_cmodel.state_dict(),
                'learning_rate': learning_rate,
                'test_acc': server.eval_test()[1],
                'model_state_dict': server.cmodel.state_dict(),
                'best_valloss': server.best_valloss,
                'best_valacc': server.best_valacc,
                }, PATH)

In [21]:
torch.load("experiments/GC/GFLAPPNP/biased_grad/I10/model_4",
            map_location=torch.device('cpu'))["test_acc"]

0.7217500000000001

In [ ]:
for ith in range(20):
    for I in [10, 20, 50]:
        file_to_open= open("experiments/GC/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(5000/I)
        exp_csbm(ith=ith, method="GFLAPPNP/", folder_path="experiments/GC/", csbm=csbm, n_train=60, n_val=20, 
                 num_communication=nc, batch_size=30,
                 learning_rate=0.5, I=I, gradient=True, noise=False,
                 Print=False, print_time=10,
                 resume=False,
                 bias=False,
                 save=True)
        print (ith, I)

0 10
0 20
0 50
1 10
1 20
1 50
2 10
2 20
2 50
3 10
3 20
3 50
4 10
4 20
4 50
5 10
5 20
5 50
6 10
6 20
6 50
7 10
7 20
